### 오픈 데이터

- 혐오 표현 데이터 참고: https://github.com/songys/AwesomeKorean_Data?tab=readme-ov-file
- K-MHaS 한국어 혐오 표현 데이터: https://github.com/adlnlp/K-MHaS
- 모델 변경 시 참고: https://github.com/Beomi/KcELECTRA

In [1]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

Using custom data configuration default
Reusing dataset kmhas (/home/u4012/.cache/huggingface/datasets/jeanlee___kmhas/default/1.0.0/17406fbed45548c92e0795df0675e21fb2a09ceaa098bd5ff58c7fdc7f8a63d4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
print(dataset)
print(dataset["train"].features)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})
{'text': Value(dtype='string', id=None), 'label': Sequence(feature=ClassLabel(num_classes=9, names=['origin', 'physical', 'politics', 'profanity', 'age', 'gender', 'race', 'religion', 'not_hate_speech'], names_file=None, id=None), length=-1, id=None)}


In [4]:
def flatten_labels(example):
    return {"label": example["label"][0] if isinstance(example["label"], list) else example["label"]}

dataset = dataset.map(flatten_labels)

  0%|          | 0/78977 [00:00<?, ?ex/s]

  0%|          | 0/8776 [00:00<?, ?ex/s]

  0%|          | 0/21939 [00:00<?, ?ex/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "beomi/KcELECTRA-small-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=9)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-small-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        add_special_tokens=True,
        stride=32,
    )
    
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": examples["label"],
    }

# 데이터셋 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/79 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

In [7]:
from transformers import Trainer, TrainingArguments

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",            # 모델 저장 경로
    eval_strategy="epoch",            # 매 에포크마다 평가
    save_strategy="epoch",            # 매 에포크마다 저장
    learning_rate=5e-5,               # 학습률
    per_device_train_batch_size=2,   # 학습 중 GPU/CPU당 처리할 배치 크기
    per_device_eval_batch_size=8,    # 평가 중 GPU/CPU당 처리할 배치 크기
    num_train_epochs=3,               # 학습할 에포크 수
    weight_decay=0.01,                # 가중치 감소 - 모델 과적합 방지
    logging_dir="./logs",             # 로그 저장 경로
    logging_steps=500,                # 몇 개의 스텝마다 로그를 기록할지
    save_total_limit=2,               # 저장할 체크포인트 개수 제한
    load_best_model_at_end=True,      # 최적의 모델 로드
    gradient_accumulation_steps=8,
    fp16=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# 'train' 데이터셋에서 'labels' 값 확인
print(set(tokenized_datasets["train"]["labels"]))

{0, 1, 2, 3, 4, 5, 6, 7, 8}


In [9]:
from transformers import Trainer

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# 모델 학습
trainer.train()

/tmp/ipykernel_3688938/812547745.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
0,0.542400,0.486197
1,0.442200,0.449231
2,0.347400,0.427744


TrainOutput(global_step=14808, training_loss=0.5450101061037332, metrics={'train_runtime': 2975.6399, 'train_samples_per_second': 79.624, 'train_steps_per_second': 4.976, 'total_flos': 1742896800483840.0, 'train_loss': 0.5450101061037332, 'epoch': 2.999924029476563})

In [10]:
# 평가
results = trainer.evaluate(tokenized_datasets["test"])
print(results)

{'eval_loss': 0.4352250397205353, 'eval_runtime': 22.6258, 'eval_samples_per_second': 969.645, 'eval_steps_per_second': 121.233, 'epoch': 2.999924029476563}


In [ ]:
# 모델과 토크나이저 저장
model.save_pretrained("./hate_speech_model")
tokenizer.save_pretrained("./hate_speech_model")

('./hate_speech_classification_model/tokenizer_config.json',
 './hate_speech_classification_model/special_tokens_map.json',
 './hate_speech_classification_model/vocab.txt',
 './hate_speech_classification_model/added_tokens.json',
 './hate_speech_classification_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 예측 수행 함수
def predict(texts):
    model = AutoModelForSequenceClassification.from_pretrained("./hate_speech_classification_model")
    tokenizer = AutoTokenizer.from_pretrained("./hate_speech_classification_model")

    # 입력 데이터 토크나이징 및 디바이스 이동
    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
        add_special_tokens=True,
        stride=32,
    )

    # 모델 추론
    with torch.no_grad():  # 예측 시에는 Gradient 계산 비활성화
        outputs = model(
            input_ids=encodings["input_ids"],
            attention_mask=encodings["attention_mask"]
        )
    logits = outputs.logits

    probabilities = torch.softmax(logits, dim=-1).numpy()

    # 클래스와 확률 추출
    predictions = []
    for text, prob in zip(texts, probabilities):
        max_prob = max(prob)  # 가장 높은 확률
        max_label = prob.argmax()  # 가장 높은 확률의 클래스(숫자)
        predictions.append({"text": text, "label": max_label, "probability": max_prob})

    return predictions